In [1]:
import keras
import keras.backend as K
from keras.models import Sequential
from keras.utils import np_utils
from keras.preprocessing.image import ImageDataGenerator
from keras.layers import Dense, Activation, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, MaxPooling2D
from keras.datasets import cifar100
from keras import regularizers, optimizers
import numpy as np

Using TensorFlow backend.


In [3]:
(x_train, y_train), (x_test, y_test) = cifar100.load_data()
# Create Validation set
x_train, x_val = x_train[:45000], x_train[45000:]
y_train, y_val = y_train[:45000], y_train[45000:]

x_train = x_train.astype('float32')
x_val = x_val.astype('float32')
x_test = x_test.astype('float32')

#z-score
mean = np.mean(x_train,axis=(0,1,2,3))
std = np.std(x_train,axis=(0,1,2,3))
x_train = (x_train-mean)/(std+1e-7)
x_val = (x_val-mean)/(std+1e-7)
x_test = (x_test-mean)/(std+1e-7)

num_classes = 100
y_train = np_utils.to_categorical(y_train,num_classes)
y_val = np_utils.to_categorical(y_val,num_classes)
y_test = np_utils.to_categorical(y_test,num_classes)

In [4]:
# e_swish_2 custom activation
def e_swish_2(x):
    return K.maximum(x*K.sigmoid(x), x*(2-K.sigmoid(x)))

In [5]:
def create(act):
# ELU NETWORK CIFAR-100 VERSION
    weight_decay = 0.0005

    model = Sequential()
    # First stack
    model.add(Conv2D(192, (5,5), padding='same', kernel_regularizer=regularizers.l2(weight_decay), input_shape=x_train.shape[1:]))
    model.add(Activation(act))
    model.add(MaxPooling2D(pool_size=(2,2)))
    
    # Second stack
    model.add(Conv2D(192, (1,1), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation(act))
    model.add(Conv2D(240, (3,3), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation(act))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.1))

    # Third stack
    model.add(Conv2D(240, (1,1), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation(act))
    model.add(Conv2D(260, (1,1), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation(act))
    model.add(Conv2D(260, (1,1), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation(act))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.2))
    
    # Fourth stack
    model.add(Conv2D(260, (1,1), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation(act))
    model.add(Conv2D(280, (1,1), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation(act))
    model.add(Conv2D(280, (1,1), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation(act))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.3))
    
    # Fifth stack
    model.add(Conv2D(280, (1,1), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation(act))
    model.add(Conv2D(300, (1,1), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation(act))
    model.add(Conv2D(300, (1,1), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation(act))
    model.add(MaxPooling2D(pool_size=(2,2)))
    model.add(Dropout(0.4))
    
    # Sixth stack
    model.add(Conv2D(300, (1,1), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation(act))
    model.add(Dropout(0.5))
    
    # Seventh and last stack
    model.add(Conv2D(100, (1,1), padding='same', kernel_regularizer=regularizers.l2(weight_decay)))
    model.add(Activation("sigmoid"))

    model.add(Flatten())
    model.summary()
    return model

In [6]:
model = create(e_swish_2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 32, 32, 192)       14592     
_________________________________________________________________
activation_1 (Activation)    (None, 32, 32, 192)       0         
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 192)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 192)       37056     
_________________________________________________________________
activation_2 (Activation)    (None, 16, 16, 192)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 16, 16, 240)       414960    
_________________________________________________________________
activation_3 (Activation)    (None, 16, 16, 240)       0         
__________

In [7]:
#data augmentation
datagen = ImageDataGenerator(
    featurewise_center=False,
    samplewise_center=False,
    featurewise_std_normalization=False,
    samplewise_std_normalization=False,
    zca_whitening=False,
    rotation_range=15,
    width_shift_range=0.1,
    height_shift_range=0.1,
    horizontal_flip=True,
    vertical_flip=False
    )
datagen.fit(x_train)

In [8]:
def schedule(x):
    return 0.01

In [9]:
#training
batch_size  = 100
epochs = 1

opt = keras.optimizers.SGD(lr=0.01, momentum=0.9, decay=0.0, nesterov=False)
lr_1 = keras.callbacks.LearningRateScheduler(schedule)
model.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
print("Finished compiling")

####################
# Network training #
####################
                     
print("Gonna fit the model")
his = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,
                          epochs=epochs,verbose=1,validation_data=(x_val,y_val), callbacks=[lr_1])
# model.save('wrn_16_8_00_'+str(i)+'.h5')
print(his.history)

Finished compiling
Gonna fit the model
Epoch 1/1
450/450 [==============================] - 348s 773ms/step - loss: 6.1127 - acc: 0.0178 - val_loss: 5.8733 - val_acc: 0.0296
{'val_loss': [5.8733166599273678], 'loss': [6.1127480687035458], 'val_acc': [0.029599999636411668], 'acc': [0.01779999966836638]}


In [10]:
model.evaluate(x_test, y_test)

10000/10000 [==============================] - 26s 3ms/step


[5.864646625518799, 0.031800000000000002]

In [13]:
# Try elu for comparisons
model_1 = create("elu")

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_29 (Conv2D)           (None, 32, 32, 192)       14592     
_________________________________________________________________
activation_29 (Activation)   (None, 32, 32, 192)       0         
_________________________________________________________________
max_pooling2d_11 (MaxPooling (None, 16, 16, 192)       0         
_________________________________________________________________
conv2d_30 (Conv2D)           (None, 16, 16, 192)       37056     
_________________________________________________________________
activation_30 (Activation)   (None, 16, 16, 192)       0         
_________________________________________________________________
conv2d_31 (Conv2D)           (None, 16, 16, 240)       414960    
_________________________________________________________________
activation_31 (Activation)   (None, 16, 16, 240)       0         
__________

In [14]:
#training
batch_size  = 100
epochs = 2

opt = keras.optimizers.SGD(lr=0.01, momentum=0.9, decay=0.0, nesterov=False)
lr_1 = keras.callbacks.LearningRateScheduler(schedule)
model_1.compile(loss="categorical_crossentropy", optimizer=opt, metrics=["accuracy"])
print("Finished compiling")

####################
# Network training #
####################
                     
print("Gonna fit the model")
his = model_1.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),steps_per_epoch=x_train.shape[0] // batch_size,
                          epochs=epochs,verbose=1,validation_data=(x_val,y_val), callbacks=[lr_1])
# model.save('wrn_16_8_00_'+str(i)+'.h5')
print(his.history)
model_1.evaluate(x_test, y_test)

Finished compiling
Gonna fit the model
Epoch 1/2
450/450 [==============================] - 258s 573ms/step - loss: 6.0020 - acc: 0.0280 - val_loss: 5.7158 - val_acc: 0.0502
Epoch 2/2
450/450 [==============================] - 257s 571ms/step - loss: 5.6279 - acc: 0.0453 - val_loss: 5.3846 - val_acc: 0.0712
{'val_loss': [5.7158360862731934, 5.3845820140838621], 'loss': [6.0020019658406571, 5.6279497167799208], 'val_acc': [0.050199999734759332, 0.071200000606477257], 'acc': [0.028022221802837317, 0.045288888555434018]}
10000/10000 [==============================] - 21s 2ms/step


[5.3836225204467771, 0.0722]